In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split


In [2]:
input_shape = (256, 256, 3)  # Input shape of the images
batch_size = 16  # Number of samples per gradient update
epochs = 2  # Number of iterations over the entire dataset

In [3]:
# Path to the dataset folders
train_folder = "/kaggle/input/airbus-ship-detection/train_v2"
test_folder = "/kaggle/input/airbus-ship-detection/test_v2"
csv_path = "/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv"

# Load the ship segmentation annotations from the CSV file
df = pd.read_csv(csv_path)

# Remove rows with missing annotations
df = df.dropna()

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Function to encode the ship segmentation masks
def encode_mask(mask):
    pixels = mask.strip().split()
    pixels = [int(p) for p in pixels]
    mask = np.zeros(input_shape[:2])
    for i in range(len(pixels) // 2):
        start = pixels[2 * i] - 1
        length = pixels[2 * i + 1]
        mask[start : start + length] = 1
    return mask

# Function to decode the ship segmentation masks
def decode_mask(mask):
    if isinstance(mask, list):
        mask = np.array(mask)
    pixels = []
    current_pixel = 0
    started = False
    for i in range(mask.shape[0]):
        if np.any(mask[i] == 1) and not started:
            pixels.append(i + 1)
            started = True
            current_pixel = 1
        elif np.any(mask[i] == 1) and started:
            current_pixel += 1
        elif np.any(mask[i] == 0) and started:
            pixels.append(current_pixel)
            started = False
            current_pixel = 0
    if started:
        pixels.append(current_pixel)
    return " ".join(str(p) for p in pixels)


# Function to load and preprocess an image
def load_image(image_id, folder):
    image_path = os.path.join(folder, image_id)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, input_shape[:2])
    image = image / 255.0
    return image

# Function to load and preprocess the masks
def load_mask(image_id):
    mask_df = df[df["ImageId"] == image_id]
    masks = np.zeros(input_shape[:2])
    for _, row in mask_df.iterrows():
        mask = encode_mask(row["EncodedPixels"])
        masks = np.maximum(masks, mask)
    masks = np.expand_dims(masks, axis=-1)
    return masks

# Function to generate a batch of images and masks
def generate_batch(image_ids, folder):
    while True:
        for batch_start in range(0, len(image_ids), batch_size):
            batch_ids = image_ids[batch_start : batch_start + batch_size]
            images = []
            masks = []
            for image_id in batch_ids:
                image = load_image(image_id, folder)
                mask = load_mask(image_id)
                images.append(image)
                masks.append(mask)
            yield np.array(images), np.array(masks)

# Generate training and validation data generators
train_image_ids = train_df["ImageId"].values
val_image_ids = val_df["ImageId"].values
train_generator = generate_batch(train_image_ids, train_folder)
val_generator = generate_batch(val_image_ids, train_folder)


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

def build_simple_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(32, 3, activation="relu", padding="same")(inputs)
    conv1 = Conv2D(32, 3, activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, activation="relu", padding="same")(pool1)
    conv2 = Conv2D(64, 3, activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, 3, activation="relu", padding="same")(pool2)
    conv3 = Conv2D(128, 3, activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, 3, activation="relu", padding="same")(pool3)
    conv4 = Conv2D(256, 3, activation="relu", padding="same")(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    conv5 = Conv2D(512, 3, activation="relu", padding="same")(pool4)
    conv5 = Conv2D(512, 3, activation="relu", padding="same")(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2D(256, 2, activation="relu", padding="same")(
        UpSampling2D(size=(2, 2))(drop5)
    )
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(256, 3, activation="relu", padding="same")(merge6)
    conv6 = Conv2D(256, 3, activation="relu", padding="same")(conv6)
    
    up7 = Conv2D(128, 2, activation="relu", padding="same")(
        UpSampling2D(size=(2, 2))(conv6)
    )
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(128, 3, activation="relu", padding="same")(merge7)
    conv7 = Conv2D(128, 3, activation="relu", padding="same")(conv7)
    
    up8 = Conv2D(64, 2, activation="relu", padding="same")(
        UpSampling2D(size=(2, 2))(conv7)
    )
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(64, 3, activation="relu", padding="same")(merge8)
    conv8 = Conv2D(64, 3, activation="relu", padding="same")(conv8)
    
    up9 = Conv2D(32, 2, activation="relu", padding="same")(
        UpSampling2D(size=(2, 2))(conv8)
    )
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(32, 3, activation="relu", padding="same")(merge9)
    conv9 = Conv2D(32, 3, activation="relu", padding="same")(conv9)
    
    outputs = Conv2D(1, 1, activation="sigmoid")(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Build the simplified U-Net model
model = build_simple_unet(input_shape)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 32  9248        ['conv2d[0][0]']                 
                                )                                                             

In [9]:
# Create a directory to save the best model weights
os.makedirs("weights", exist_ok=True)
checkpoint_path = "/kaggle/input/weights-for-airbus-ship/semantic_segmentation_weights.h5"

In [5]:
# Define a checkpoint callback to save the best model weights
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode="min",
    save_weights_only=True,
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_image_ids) // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_image_ids) // batch_size,
    callbacks=[checkpoint_callback],
)


Epoch 1/2


2023-06-01 19:02:24.309275: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4086/4086 [==============================] - ETA: 0s - loss: 4.4146 - accuracy: 0.9982
Epoch 1: val_loss improved from inf to 0.00971, saving model to weights/semantic_segmentation_weights.h5
4086/4086 [==============================] - 2941s 713ms/step - loss: 4.4146 - accuracy: 0.9982 - val_loss: 0.0097 - val_accuracy: 0.9987
Epoch 2/2
4086/4086 [==============================] - ETA: 0s - loss: 0.0106 - accuracy: 0.9986
Epoch 2: val_loss improved from 0.00971 to 0.00968, saving model to weights/semantic_segmentation_weights.h5
4086/4086 [==============================] - 2676s 654ms/step - loss: 0.0106 - accuracy: 0.9986 - val_loss: 0.0097 - val_accuracy: 0.9987


In [ ]:
from IPython.display import clear_output

# Load the best model weights
model.load_weights(checkpoint_path)

# Function to predict masks for test images
def predict_masks(image_ids, folder):
    masks = []
    for image_id in image_ids:
        clear_output(wait=True)
        image = load_image(image_id, folder)
        mask = model.predict(np.expand_dims(image, axis=0))[0]
        mask = (mask > 0.5).astype(np.uint8)
        masks.append(mask)
    return masks

# Load the test image IDs
test_image_ids = os.listdir(test_folder)

# Predict masks for test images
test_masks = predict_masks(test_image_ids, test_folder)
        
# Save the predicted masks as submission
submission = pd.DataFrame({"ImageId": test_image_ids, "EncodedPixels": ""})
for i, image_id in enumerate(test_image_ids):
    mask = test_masks[i]
    encoded_pixels = decode_mask(mask)
    submission.loc[submission["ImageId"] == image_id, "EncodedPixels"] = encoded_pixels
submission.to_csv("submission.csv", index=False)
